In [2]:
%%time
import os
! pip install -qq -U langchain tiktoken pypdf chromadb faiss-gpu
! pip install -qq -U transformers InstructorEmbedding sentence_transformers
! pip install -qq -U accelerate bitsandbytes xformers einops

CPU times: user 61.9 ms, sys: 72.7 ms, total: 135 ms
Wall time: 13.5 s


In [8]:
pip install pyopenssl --upgrade

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [10]:
# pip install --no-cache fastapi

In [11]:
# !pip install typing-inspect 

In [12]:
# pip install typing-extensions==4.5.0

In [13]:
# !pip install typing_extensions  

In [14]:
# pip install unstructured

In [20]:
# pip install langchain --upgrade

In [21]:
# !pip -q install langchain==0.0.018
# !pip show langchain

In [22]:
# !pip install pydantic 

In [2]:
import langchain

In [18]:
import warnings
warnings.filterwarnings("ignore")

import textwrap
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline,LlamaTokenizer,BitsAndBytesConfig
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.document_loaders import UnstructuredPDFLoader
import torch
import langchain
print(langchain.__version__)

0.0.230


In [4]:
loader = UnstructuredPDFLoader("story.pdf")
transcript = loader.load()

In [5]:
## SPLIT THE PDF FILE#####

text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=50)
texts = text_splitter.split_documents(transcript) 
len(texts)

9

In [6]:
###Embedding and convert chunks to Vectordb using chroma##

In [7]:
%%time
### this takes ~35 min to run
from langchain.embeddings import HuggingFaceInstructEmbeddings

persist_directory = 'story'

### download embeddings model
instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl")

### create embeddings and DB
vectordb = Chroma.from_documents(documents=texts,
                                 embedding=instructor_embeddings,
                                 persist_directory=persist_directory
                                )



### persist Chroma database
vectordb.persist()



load INSTRUCTOR_Transformer
max_seq_length  512
CPU times: user 3min 20s, sys: 1min 17s, total: 4min 37s
Wall time: 23.3 s


In [8]:
# %%time

load_directory = 'story'

### download embeddings model
instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl"
                                                     )

vectordb = Chroma(persist_directory=load_directory, embedding_function=instructor_embeddings)



load INSTRUCTOR_Transformer
max_seq_length  512


In [9]:
## use falcon LLM model or any other model##

In [29]:
quantization_config = BitsAndBytesConfig(llm_int8_threshold=200.0)
tokenizer = AutoTokenizer.from_pretrained("h2oai/h2ogpt-gm-oasst1-en-2048-falcon-7b-v2")
        
model = AutoModelForCausalLM.from_pretrained("h2oai/h2ogpt-gm-oasst1-en-2048-falcon-7b-v2",
                                                     load_in_8bit=True,
                                                     device_map='auto',
                                                     torch_dtype=torch.bfloat16,
                                                     low_cpu_mem_usage=True,
                                                     trust_remote_code=True,
                                                     quantization_config=quantization_config
                                                    )
max_len = 4098
task = "text-generation"
T = 0     


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [15]:
## Pass llm model into the huggingface Pipeline###

In [30]:
pipe = pipeline(
    task=task,
    model=model, 
    tokenizer=tokenizer, 
    max_length=max_len,
    temperature=T,
    top_p=0.95,
    repetition_penalty=1.15
)
llm = HuggingFacePipeline(pipeline=pipe)

The model 'RWForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PLBartForCausalLM', 'ProphetNetForCausalLM', 'QDQBertLMHeadModel', 'ReformerModelWithLMHead', 'RemBertForCausalLM', 'RobertaForCausalLM', '

In [17]:
## pass full pipeline into the QA chain####


In [31]:
retriever = vectordb.as_retriever(search_kwargs={"k": 3, "search_type" : "similarity"})

qa_chain = RetrievalQA.from_chain_type(llm=llm, 
                                       chain_type="stuff", 
                                       retriever=retriever, 
                                       return_source_documents=True,
                                       verbose=True)

In [21]:


def final_text(text, width=300):
    lines = text.split('\n')
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]
    wrapped_text = '\n'.join(wrapped_lines)
    return wrapped_text


In [27]:
def final_llm_response(llm_final_response):
    print(final_text(llm_final_response['result']))

def final_answer(question):
    llm_final_response = qa_chain(question)
    answer = final_llm_response(llm_final_response)
    return answer

In [ ]:
%%time
question = "When was the first day of Donna's holiday?"
final_answer(question)



> Entering new  chain...


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
